In [1]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [2]:
arcpy.ResetEnvironments()
gdb_name = "Summarize_Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [3]:
data = ["in_data/Pr_Lg_Fire.shp", "in_data/Pct_Cg_Lg_Fi.shp",
        "in_data/WUI_Intermix.shp", "in_data/WUI_Interface.shp", 
        "out_data/lu/LU_merged.shp", "out_data/ru/Reporting_Units_APN.shp"]

for v in data:
    path = os.path.join(THIS_FOLDER, "{}".format(v))
    name = v.rsplit('/', 1)[-1].split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)

In [4]:
arcpy.analysis.SummarizeWithin("Reporting_Units_APN_rpj", "Pr_Lg_Fire_rpj", 
                               "A_RU_1", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_1
1,


In [5]:
arcpy.analysis.SummarizeWithin("A_RU_1", "Pct_Cg_Lg_Fi_rpj", 
                               "A_RU_2", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_2
1,


In [6]:
arcpy.analysis.SummarizeWithin("A_RU_2", "WUI_Intermix_rpj", 
                               "A_RU_3", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.analysis.SummarizeWithin("A_RU_3", "WUI_Interface_rpj", 
                               "A_RU_4", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_4
1,


In [7]:
arcpy.analysis.SummarizeWithin("A_RU_4", "LU_Merged_rpj", 
                               "A_RU_5", "KEEP_ALL", "LU_Cat Max", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", "LU_Cat", "ADD_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_5
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\LU_Cat_Summary


In [8]:
fieldObs = arcpy.ListFields("A_RU_5")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames) 

with arcpy.da.UpdateCursor("A_RU_5", fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)

del curU

In [9]:
arcpy.AddField_management("A_RU_5", "WUI_Interm", "LONG")
arcpy.AddField_management("A_RU_5", "WUI_Interf", "LONG")

arcpy.CalculateField_management("A_RU_5", "WUI_Interm", 
                                'math.ceil(!SUM_Area_SQUAREKILOMETERS!)', "PYTHON3")
arcpy.CalculateField_management("A_RU_5", "WUI_Interf", 
                                'math.ceil(!SUM_Area_SQUAREKILOMETERS_1!)', "PYTHON3")

arcpy.DeleteField_management("A_RU_5", 
                             ['SUM_Area_SQUAREKILOMETERS', 'SUM_Area_SQUAREKILOMETERS_1'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_5'>

In [10]:
arcpy.AlterField_management('A_RU_5', 
                            'MEAN_gridcode', 'Pr_Lg_Fr', 'Pr_Lg_Fr')
arcpy.AlterField_management('A_RU_5', 
                            'MEAN_gridcode_1', 'Pc_Cg_LgFi', 'Pc_Cg_LgFi')
arcpy.AlterField_management('A_RU_5', 
                            'Majority_LU_Cat', 'Mj_LU_Cat', 'Mj_LU_Cat')
arcpy.DeleteField_management("A_RU_5", 
                             ['SUM_Area_SQUAREKILOMETERS_12', 'MAX_LU_Cat', 'Minority_LU_Cat', 'Join_ID_1', 'Shape_Leng'])

fieldObs = arcpy.ListFields("A_RU_5")  

fieldNames = []  
for field in fieldObs: 
    print(field.name)
    fieldNames.append(field.name)  
del fieldObs

OBJECTID
Shape
Join_ID
apn
Pr_Lg_Fr
Pc_Cg_LgFi
Shape_Length
Shape_Area
Mj_LU_Cat
WUI_Interm
WUI_Interf


In [11]:
dir_path = os.path.join(THIS_FOLDER, "out_data/eems")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("A_RU_5", 
                                        os.path.join(THIS_FOLDER, "out_data/eems"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\eems'>

In [13]:
gdf = gpd.read_file(os.path.join(THIS_FOLDER, "out_data/eems/A_RU_5.shp"))
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18180 entries, 0 to 18179
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Join_ID     18180 non-null  int64   
 1   apn         18179 non-null  object  
 2   Pr_Lg_Fr    18180 non-null  float64 
 3   Pc_Cg_LgFi  18180 non-null  float64 
 4   Shape_Leng  18180 non-null  float64 
 5   Shape_Area  18180 non-null  float64 
 6   Mj_LU_Cat   18180 non-null  object  
 7   WUI_Interm  18180 non-null  int64   
 8   WUI_Interf  18180 non-null  int64   
 9   geometry    18180 non-null  geometry
dtypes: float64(4), geometry(1), int64(3), object(2)
memory usage: 1.4+ MB


In [14]:
gdf.to_csv("out_data/eems/A_RU_5.csv", index=False)